In [1]:
import vk_api, json
from vk_api.longpoll import VkLongPoll, VkEventType
from vk_api.keyboard import VkKeyboard, VkKeyboardColor
from vk_api.bot_longpoll import VkBotLongPoll, VkBotEventType
import requests
import sqlite3
from collections import defaultdict
from vk_api.utils import get_random_id
import time

In [2]:
token = '9d662b711c8bb91e64dbafc5cb5088218eab3b578758d4a2c878b4e2ce11d8a6e3bc869d65ab99708e1ef'
group_id = '201615991'
version = '5.103'
group_name_short = 'bec204'

In [3]:
session = vk_api.VkApi(token=token)
longpoll = VkBotLongPoll(session, group_id)

In [4]:
def send_message_to_user (user_id, peer_id, message, keyboard=None): #сделать проще!
    post = {
        "user_id": user_id,
        "random_id": get_random_id(),
        "peer_id": peer_id,
        "message": message,
        "random_id": 0
    }
    if keyboard != None:
        post = {
        "user_id": user_id,
        "random_id": get_random_id(),
        "peer_id": peer_id,
        "message": message,
        "random_id": 0,
        "keyboard" : keyboard.get_keyboard()
        }
    session.method('messages.send', post)

In [5]:
def send_message_to_chat (chat_id, peer_id, message, keyboard=None): #сделать проще!
    post = {
        "chat_id": chat_id,
        "random_id": get_random_id(),
        "peer_id": peer_id,
        "message": message,
        "random_id": 0
    }
    if keyboard != None:
        post = {
        "chat": chat_id,
        "random_id": get_random_id(),
        "peer_id": peer_id,
        "message": message,
        "random_id": 0,
        "keyboard" : keyboard.get_keyboard()
        }
    session.method('messages.send', post)

In [6]:
def get_conversations(offset, count, filt):
    post = {
        "offset": offset,
        "count": count,
        "filter": filt
    }
    session.method("messages.getConversations", post)

In [7]:
def get_Conversations (peer_ids): 
    post = {
        "peer_ids": peer_ids
    }
    session.method("messages.getConversations", post)

In [8]:
def vk_download(method, parameters, fields=''):
    url = 'https://api.vk.com/method/' + method + '?' + parameters + '&fields=' + fields + '&access_token=' + token + '&v=' + version
    response = requests.get(url)
    infa = response.json()
    return infa

In [9]:
# подключаем базу данных
conn = sqlite3.connect('chats_hse.db')

# курсор для работы с таблицами
cursor = conn.cursor()

try:
    # sql запрос для создания таблицы
    query = "CREATE TABLE \"chats\" (\"ID\" INTEGER UNIQUE, \"chat\" TEXT, PRIMARY KEY (\"ID\"))"
    # исполняем его –> ура, теперь у нас есть таблица, куда будем все сохранять!
    cursor.execute(query)
except:
    pass

In [10]:
#добавить группу в БД
def add_group(peer, text):
    with sqlite3.connect('chats_hse.db') as con:
        cursor = con.cursor()
        cursor.execute('INSERT INTO chats (ID, chat) VALUES (?, ?)',
                       (peer, text))
        con.commit()


In [11]:
#вывод списка групп
def normal(st_group):
    groups = defaultdict()
    for elem in st_group:
        chat_id = elem[0]
        group = elem[1]
        groups[group] = chat_id
    return groups
    
def choose_group():
    with sqlite3.connect('chats_hse.db') as con:
        cursor = con.cursor()
        cursor.execute('SELECT * FROM chats')
        groups = normal(cursor.fetchall())
    return groups

In [ ]:
#изменение название группы
def change_name(chat, peer_id):
    with sqlite3.connect('chats_hse.db') as con:
        cursor = con.cursor()
        cursor.execute('UPDATE chats SET chat=={}'.format(chat) + ' WHERE ID=={}'.format(peer_id))
        con.commit()

In [ ]:
#удаление беседы
def delete(text):
    with sqlite3.connect('chats_hse.db') as con:
        cursor = con.cursor()
        cursor.execute('DELETE FROM chats WHERE chat=={}'.format(text))
        con.commit()
    

In [ ]:
while True:
    
    try:
        for event in longpoll.listen():
            if event.type == VkBotEventType.MESSAGE_NEW:
                text = event.object['text'].lower()
                peer_id = event.obj.peer_id
        
                if event.from_user:#Если написали в ЛС
                    user_id = event.object['from_id']
                    
                    if text in ['начать', 'продолжить работу']:
                        keyboard_start = VkKeyboard(one_time = False, inline = True)
                        keyboard_start.add_button('Удалить беседу', VkKeyboardColor.NEGATIVE) #сделать функцию
                        keyboard_start.add_line()
                        keyboard_start.add_button('Отправить сообщение', VkKeyboardColor.SECONDARY)

                        send_message_to_user(user_id, peer_id, 'Что сделать?', keyboard_start)

#кнопка "отправить сообщение"
                    elif text == 'отправить сообщение':
                        chosen_groups = []
                        keyboard_choose_group = VkKeyboard(one_time = False)
                        groups = choose_group()
                        for elem in groups:
                            keyboard_choose_group.add_button(elem)
#                             keyboard_choose_group.add_line()
                        keyboard_choose_group.add_button('Всё выбрано. Ввести сообщение')
                        send_message_to_user(user_id, peer_id, 'Выберите группу', keyboard_choose_group)
                        for event in longpoll.listen():
                            if event.type == VkBotEventType.MESSAGE_NEW:
                                text = event.object['text'].lower()
                                peer_id = event.obj.peer_id
                                if text in list(groups.keys()):
                                    chosen_groups.append(groups[text])
                                    send_message_to_user(user_id, peer_id, 'Ещё одну?', keyboard_choose_group)
                    
                                elif text == 'всё выбрано. ввести сообщение':
                                    send_message_to_user(user_id, peer_id, 'Введите сообщение')
                                    for event in longpoll.listen():
                                        if event.type == VkBotEventType.MESSAGE_NEW:
                                            text = event.object['text'].lower()
                                            user_id = event.object['from_id']

                                            for elem in chosen_groups:
                                                send_message_to_chat(elem - 2000000000, elem, text)
                                                timeout(0.1)
                                            send_message_to_user(user_id, peer_id, 'Рассылка завершена')
                                        break
                        
                        keyboard_end = VkKeyboard(one_time = True)
                        keyboard_end.add_button('Пока всё')
                        keyboard_end.add_button('Продолжить работу')
                        send_message_to_user(user_id, peer_id, 'Что-то ещё?', keyboard_end)
                    
                    elif text == 'Пока всё':
                        send_message_to_user(user_id, peer_id, 'Хорошо. Если захотите продолжить работу, введите "начать"')
                    
# кнопка 'удалить беседу'
                    elif text == 'Удалить беседу':
                        chosen_groups = []
                        keyboard_choose_group = VkKeyboard(one_time = False)
                        groups = choose_group()
                        for elem in groups:
                            keyboard_choose_group.add_button(elem)
#                             keyboard_choose_group.add_line()
                        keyboard_choose_group.add_button('Назад')
                        send_message_to_user(user_id, peer_id, 'Выберите группу, беседу которой хотите удалить', keyboard_choose_group)
        #получаем название удаляемой группы
                        for event in longpoll.listen():
                            if event.type == VkBotEventType.MESSAGE_NEW:
                                group = event.object['text']
                                peer_id = event.obj.peer_id
                                if group in list(groups.keys()):
                                    keyboard_choice = VkKeyboard(one_time = True)
                                    keyboard_choice.add_button('Да', VkKeyboardColor.POSITIVE)
                                    keyboard_choice.add_button('Отмена', VkKeyboardColor.SECONDARY)
                                    send_message_to_user(user_id, peer_id, 'Вы уверены? Действие нелья будет отменить', keyboard_choice)
                                    #получаем подтверждение удаления
                                    for event in longpoll.listen():
                                        if event.type == VkBotEventType.MESSAGE_NEW:
                                            text = event.object['text'].lower()
                                            peer_id = event.obj.peer_id
                                            if text == 'да':
                                                delete(group)
                                                send_message_to_user(user_id, peer_id, 'Беседа удалена')
                                            elif text == 'отмена':
                                                send_message_to_user(user_id, peer_id, 'Выберите группу, беседу которой хотите удалить', keyboard_choose_group) 
                                            break
                                        
                                    
                                elif text == 'Назад':
                                    send_message_to_user(user_id, peer_id, 'Что сделать?', keyboard_start)
                     
                                break

                elif event.from_chat:
                    chat_id = event.chat_id
                    if text == 'регистрация':
                        send_message_to_chat(chat_id, peer_id, 'Введите название своей группы')
                        for event in longpoll.listen():
                            if event.type == VkBotEventType.MESSAGE_NEW:
                                text = event.object['text']
                                peer_id = event.obj.peer_id
                                add_group(peer_id, text)
                                send_message_to_chat(chat_id, peer_id, 'Принято')
                                break
                                
                    elif text == 'Изменить название группы':
                        send_message_to_chat(chat_id, peer_id, 'Введите новое название своей группы')
                        
                        for event in longpoll.listen():
                            if event.type == VkBotEventType.MESSAGE_NEW:
                                chat_id = event.chat_id
                                peer_id = event.obj.peer_id
                                text = event.object['text']
                                change_name(text, peer_id)
                                send_message_to_chat(chat_id, peer_id, 'Название изменено')
                                break
                        
#             if event.type == VkBotEventType.MESSAGE_EVENT: #callback-кнопки   
#                 print(event)

                
    except requests.exceptions.ReadTimeout as timeout:
        continue          
                
#         elif event.from_chat: #Если написали в Беседе
#             chat_id = event.chat_id
#             if text = 'Зарегистрировать беседу': #ветка с регистрацией в БД
                

2000000001
2000000003


In [ ]:
# if event_chat.type

In [ ]:
event.type == VkBotEventType.MESSAGE_EVENT